# Часть 1. Проверка гипотезы в Python и составление аналитической записки

## Проект: Анализ различий в пользовательской активности между Москвой и Санкт-Петербургом

### Цель проекта

Проверить, существует ли статистически значимая разница в среднем времени активности пользователей приложения «Яндекс.Книги» между жителями Москвы и Санкт-Петербурга.

### Задачи проекта

- Загрузить и предобработать данные: проверить наличие дубликатов и пропусков, привести данные к корректным типам.
- Разделить пользователей по городам: Москва и Санкт-Петербург.
- Сравнить размеры групп и рассчитать описательные статистики: среднее, медиана, стандартное отклонение.
- Построить визуализации распределения времени активности.
- Провести односторонний t-тест для двух независимых выборок.
- Оценить результат теста и сделать статистически обоснованные выводы.
- Предложить возможные объяснения наблюдаемого результата.

### Описание данных

**Таблица пользователей `yandex_knigi_data.csv`**  
Содержит агрегированную информацию о пользователях из двух городов и их активности в приложении. Структура данных:

- `user_id` — идентификатор пользователя;
- `city` — город проживания (`Москва` или `Санкт-Петербург`);
- `total_time` — суммарное время активности пользователя в приложении (в часах).

**Дополнительные источники данных**
- `group` — группа пользователя;
- `ab_test` — название A/B-теста;
- `device` — устройство, с которого происходила регистрация;
- `event_dt` — дата и время события;
- `event_name` — тип события;
- `details` — дополнительные данные о событии.

### Формулировка гипотезы

- **Нулевая гипотеза ($H_0$):** среднее время активности пользователей из Санкт-Петербурга **не больше**, чем у пользователей из Москвы:  
  $H_0: \mu_{\text{СПб}} \leq \mu_{\text{Москва}}$
  
- **Альтернативная гипотеза ($H_1$):** среднее время активности пользователей из Санкт-Петербурга **больше**, чем у пользователей из Москвы:  
  $H_1: \mu_{\text{СПб}} > \mu_{\text{Москва}}$

Для проверки гипотезы будет использоваться односторонний t-тест для двух независимых выборок с уровнем статистической значимости $\alpha = 0.05$.

### Этапы исследования

1. **Загрузка и первичный анализ данных**
2. **Проверка на дубликаты и пропущенные значения**
3. **Разделение данных на группы по городу**
4. **Анализ распределений и описательных статистик**
5. **Проверка статистических гипотез**
6. **Интерпретация результатов и формулировка выводов**
7. **Рекомендации и возможные объяснения**


## 1. Загрузка данных и знакомство с ними

Данные пользователей из Москвы и Санкт-Петербурга c их активностью (суммой часов чтения и прослушивания) из файла `/datasets/yandex_knigi_data.csv`.

In [1]:
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('/datasets/yandex_knigi_data.csv')

In [3]:
df.head()

,Unnamed: 0,city,puid,hours
0,0,Москва,9668,26.167776
1,1,Москва,16598,82.111217
2,2,Москва,80401,4.656906
3,3,Москва,140205,1.840556
4,4,Москва,248755,151.326434


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8784 entries, 0 to 8783
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  8784 non-null   int64  
 1   city        8784 non-null   object 
 2   puid        8784 non-null   int64  
 3   hours       8784 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 274.6+ KB


In [5]:
df.duplicated().sum()

0

In [6]:
duplicate_users = df['puid'].duplicated()

In [7]:
df['puid'].duplicated().sum()

244

In [8]:
df[df['puid'].duplicated(keep=False)].sort_values(by='puid')

,Unnamed: 0,city,puid,hours
35,35,Москва,2637041,10.317371
6247,6247,Санкт-Петербург,2637041,3.883926
134,134,Москва,9979490,32.415573
6274,6274,Санкт-Петербург,9979490,1.302997
145,145,Москва,10597984,42.931506
...,...,...,...,...
6195,6195,Москва,1130000020425037,0.310556
8775,8775,Санкт-Петербург,1130000023864516,14.384722
6202,6202,Москва,1130000023864516,142.830085
6210,6210,Москва,1130000028554332,11.277554


In [9]:
# Сохраняем id пользователей с дублирующимися сессиями
user_ids = df[df.duplicated(subset=['puid',])]['puid'].to_list()

In [10]:
# Проверим долю дублирующихся пользователей
100 * len(user_ids) / df['puid'].nunique()

2.857142857142857

In [11]:
# Сохраням изначальное количество строк df
x = df.shape[0]

In [12]:
# Удаляем дубл пользователей df
df = df[~df['puid'].isin(user_ids)].reset_index(drop=True)
df.shape[0]

8296

In [13]:
x - len(df)

488

В исходном датафрейме содержалось 8784 записи. В ходе анализа были выявлены 244 пользователя с повторяющимися puid, что составляет примерно 2.86% от общего числа уникальных пользователей. 

Это пользлователи которые попали в обе группы, а именно один уникальный юзер пользовался приложением и в Москве и в Санкт-Петербурге.

Для обеспечения корректности анализа все записи, относящиеся к дублирующимся puid, были полностью удалены. В результате очистки удалено 488 строк, и финальный размер датафрейма составил 8296 записей.

Это позволяет исключить искажение статистических показателей, вызванное многократным учетом активности одних и тех же пользователей.

## Проверка гипотезы в Python

Гипотеза звучит так: пользователи из Санкт-Петербурга проводят в среднем больше времени за чтением и прослушиванием книг в приложении, чем пользователи из Москвы.

- Нулевая гипотеза H₀: Средняя активность пользователей в часах в двух группах (Москва и Санкт-Петербург) не различается.

- Альтернативная гипотеза H₁: Средняя активность пользователей в Санкт-Петербурге больше, и это различие статистически значимо.

Выбор теста
Для проверки гипотезы о разнице средних значений между двумя независимыми группами — пользователями из Москвы и Санкт-Петербурга — был выбран t-тест.

Почему t-тест:

- Цель — сравнение средних значений между двумя группами (а не медиан или рангов).
- Пользователи из Москвы и Санкт-Петербурга не пересекаются. Это означает, что данные независимы — одно из условий применения t-теста.
- Размер выборки достаточно большой (в обеих группах более 30 наблюдений), что делает t-тест устойчивым даже при умеренном нарушении нормальности.
- t-тест чувствительнее, чем непараметрические альтернативы.
- При этом данные количественные и непрерывные, что соответствует условиям t-теста.

In [14]:
# Выделим группы
spb = df[df['city'] == 'Санкт-Петербург']['hours']
msk = df[df['city'] == 'Москва']['hours']

In [15]:
# t-тест
t_stat, p_value = stats.ttest_ind(spb, msk, alternative='greater')

print(f"t-статистика: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

t-статистика: 0.4499
p-value: 0.3264


Вывод
Результаты t-теста для сравнения средней активности (в часах) между пользователями из Санкт-Петербурга и Москвы:

t-статистика: 0.4499

p-value: 0.3264

При уровне значимости α = 0.05 мы не отвергаем нулевую гипотезу. Это означает, что статистически значимых различий в средней активности между пользователями из Санкт-Петербурга и Москвы не выявлено. Мы не можем утверждать, что пользователи из Санкт-Петербурга проводят в приложении больше времени, чем пользователи из Москвы.

## Аналитическая записка

Метод:
Применим t-тест Стьюдента для независимых выборок с односторонней альтернативной гипотезой.
Уровень статистической значимости — 0.05.

Результаты теста:

t-статистика: 0.4499

p-value: 0.3264

Вывод:

Так как p-value = 0.3264 > 0.05, мы не отвергаем нулевую гипотезу.

Это означает, что нет статистически значимых доказательств того, что пользователи из Санкт-Петербурга проводят в среднем больше времени в приложении по сравнению с пользователями из Москвы.


Возможные объяснения:
Поведение пользователей схоже в двух городах, несмотря на различия в LTV. Пользователи могут платить одинаково, но использовать сервис по-разному (например, подписка — ради нескольких любимых книг).

Разница в LTV между городами может быть связана не со временем активности, а, например, с длительностью подписки.

----

# Часть 2. Анализ результатов A/B-тестирования

Контекст задачи: 
Интернет магазин BitMotion Kit, в котором продаются геймифицированные товары для тех, кто ведёт здоровый образ жизни. Есть своя целевая аудитория, даже появились хиты продаж: эспандер со счётчиком и напоминанием, так и подстольный велотренажёр с Bluetooth.

В будущем компания хочет расширить ассортимент товаров. Но перед этим нужно решить одну проблему. Интерфейс онлайн-магазина слишком сложен для пользователей — об этом говорят отзывы.

Чтобы привлечь новых клиентов и увеличить число продаж, владельцы магазина разработали новую версию сайта и протестировали его на части пользователей. По задумке, это решение доказуемо повысит количество пользователей, которые совершат покупку.

Задача — провести оценку результатов A/B-теста.

## Цели исследования.



**Цели исследования**

Оценить влияние новой версии сайта интернет-магазина BitMotion Kit на поведение пользователей.

Проверить, увеличивает ли новая версия сайта количество пользователей, совершивших покупку.

Оценить корректность проведения A/B-теста (распределение пользователей, пересечения и прочее).


## 2. Загрузка данных и оценка целостности.


In [16]:
participants = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_participants.csv')
events = pd.read_csv('https://code.s3.yandex.net/datasets/ab_test_events.zip',
                     parse_dates=['event_dt'], low_memory=False)

In [17]:
participants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14525 entries, 0 to 14524
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  14525 non-null  object
 1   group    14525 non-null  object
 2   ab_test  14525 non-null  object
 3   device   14525 non-null  object
dtypes: object(4)
memory usage: 454.0+ KB


In [18]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787286 entries, 0 to 787285
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     787286 non-null  object        
 1   event_dt    787286 non-null  datetime64[ns]
 2   event_name  787286 non-null  object        
 3   details     249022 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 24.0+ MB


In [19]:
participants['ab_test'].unique()

array(['interface_eu_test', 'recommender_system_test'], dtype=object)

In [20]:
participants['user_id'].duplicated().sum()

887

## 3. По таблице `ab_test_participants` оцениваем корректность проведения теста:

   3\.1 Выделить пользователей, участвующих в тесте, и проверить:

   - соответствие требованиям технического задания,

   - равномерность распределения пользователей по группам теста,

   - отсутствие пересечений с конкурирующим тестом (нет пользователей, участвующих одновременно в двух тестовых группах).

In [21]:
participants['group'].unique()

array(['B', 'A'], dtype=object)

In [22]:
# Количество пользователей
interface_test = participants[participants['ab_test'] == 'interface_eu_test']
interface_test['group'].value_counts()

B    5467
A    5383
Name: group, dtype: int64

In [23]:
# Равномерность распределения пользователей по группам
interface_test['group'].value_counts(normalize=True)

B    0.503871
A    0.496129
Name: group, dtype: float64

In [24]:
# Есть ли пересечения с другими тестами
# Все пользователи из interface_eu_test
interface_test_users = set(interface_test['user_id'])

# Пользователи из других тестов
other_tests = participants[participants['ab_test'] != 'interface_eu_test']
other_test_users = set(other_tests['user_id'])

# Пересечения
intersecting_users = interface_test_users.intersection(other_test_users)
print(f"Число пользователей, участвующих одновременно в нескольких тестах: {len(intersecting_users)}")

Число пользователей, участвующих одновременно в нескольких тестах: 887


Этих пользователей мы оставляем, потому что 
- нет информации, что другие тесты затрагивали пользовательский интерфейс, и что этот может искажать результаты.
- Удаляя пользователей, мы сокращаем размер выборки — в нашем случае, на 887 пользователей.Это может снизить чувствительность теста и привести к ситуации, когда реальный эффект не будет замечен.
- Учитывая их в тесте, мы приближаем эксперимент к реальным условиям, что делает выводы более прикладными и устойчивыми к "шуму".

3\.2 Проанализируем данные о пользовательской активности по таблице `ab_test_events`:

- оставляем только события, связанные с участвующими в изучаемом тесте пользователями;

In [25]:
interface_test_events = events[events['user_id'].isin(interface_test_users)]

In [26]:
interface_test_events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79715 entries, 64672 to 780371
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     79715 non-null  object        
 1   event_dt    79715 non-null  datetime64[ns]
 2   event_name  79715 non-null  object        
 3   details     21075 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 3.0+ MB


- определяем горизонт анализа: рассчитываем время (лайфтайм) совершения события пользователем после регистрации и оставьте только те события, которые были выполнены в течение первых семи дней с момента регистрации;

In [27]:
# Получаем даты регистрации пользователей
registrations = interface_test_events[interface_test_events['event_name'] == 'registration']
registrations = registrations[['user_id', 'event_dt']].rename(columns={'event_dt': 'registration_dt'})

In [28]:
# Объединим события с датой регистрации
interface_test_events = interface_test_events.merge(registrations, on='user_id', how='left')

# Добавим колонку с лайфтаймом — разницей между событием и регистрацией в днях
interface_test_events['lifetime_days'] = (interface_test_events['event_dt'] - interface_test_events['registration_dt']).dt.days

In [29]:
# Сохраняем только события с лайфтаймом до 7 дней (включительно)
interface_test_events_7d = interface_test_events[interface_test_events['lifetime_days'] <= 7]

Оцениваем достаточность выборки для получения статистически значимых результатов A/B-теста. Заданные параметры:

- базовый показатель конверсии — 30%,

- мощность теста — 80%,

- достоверность теста — 95%.

In [30]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# параметры
baseline = 0.3          # базовая конверсия
target = 0.35           # ожидаемая конверсия после изменений
alpha = 0.05            # уровень значимости
power = 0.8             # мощность теста

# размер эффекта
effect_size = proportion_effectsize(baseline, target)

# расчет объема выборки
analysis = NormalIndPower()
sample_size = analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1, alternative='two-sided')

print(f"Необходимое количество пользователей в каждой группе: {round(float(sample_size))}")


Необходимое количество пользователей в каждой группе: 1376


- рассчитываем для каждой группы количество посетителей, сделавших покупку, и общее количество посетителей.

In [31]:
# Общее количество пользователей в каждой группе
total_users_by_group = interface_test[['user_id', 'group']].drop_duplicates().groupby('group')['user_id'].count()
total_users_by_group

group
A    5383
B    5467
Name: user_id, dtype: int64

In [32]:
interface_test_events = interface_test_events.merge(interface_test, on='user_id', how='left')

In [33]:
# Количество пользователей, совершивших покупку в каждой группе
purchases_by_group = (
    interface_test_events[interface_test_events['event_name'] == 'purchase']
    .groupby('group')['user_id']
    .nunique()
)
purchases_by_group

group
A    1766
B    1931
Name: user_id, dtype: int64

- Делаем предварительный общий вывод об изменении пользовательской активности в тестовой группе по сравнению с контрольной.

In [34]:
# Расчёт конверсии для каждой группы
conversion_by_group = (purchases_by_group / total_users_by_group * 100).round(2)
conversion_by_group

group
A    32.81
B    35.32
Name: user_id, dtype: float64

In [35]:
uplift_pp = (conversion_by_group['B'] - conversion_by_group['A']).round(2)
uplift_pp

2.51

## 4. Оценка результатов A/B-тестирования:

- Проверяем изменение конверсии подходящим статистическим тестом, учитывая все этапы проверки гипотез.

H₀ (нулевая гипотеза): Конверсии в группах A и B равны. То есть, разница в конверсии между группами отсутствует.

H₁ (альтернативная гипотеза): Конверсии в группах A и B различаются. То есть, разница в конверсии между группами существует.

In [36]:
from statsmodels.stats.proportion import proportions_ztest

# Количество успехов (покупок)
successes = [purchases_by_group['A'], purchases_by_group['B']]

# Общее количество пользователей в каждой группе
totals = [total_users_by_group['A'], total_users_by_group['B']]

# Проведение z-теста
z_stat, p_value = proportions_ztest(count=successes, nobs=totals, alternative='two-sided')

print(f'z-статистика: {z_stat:.4f}')
print(f'p-value: {p_value:.4f}')


z-статистика: -2.7625
p-value: 0.0057


## Выводы по результатам A/B-тестирования

Результаты статистического теста:
- z-статистика: -2.7625
- p-value: 0.0057


Так как p-value < 0.05, мы отвергаем нулевую гипотезу.
Это означает, что разница в конверсии между группами A и B статистически значима.
Конверсия в группе B (тестовая) выше, чем в группе A (контрольной):
- Группа A: 32.81%
- Группа B: 35.32%

Аплифт: +2.51 п.п. (процентных пункта)

### Ожидаемый эффект по ТЗ:
- В техническом задании указано, что ожидаемый прирост должен составлять не менее 3 п.п.
- Фактический прирост составил только 2.51 п.п.

**Интерпретация:**

- Несмотря на статистически значимый рост, фактический прирост не достиг порогового значения, указанного в гипотезе (3 п.п.). То есть бизнес-гипотеза не подтвердилась полностью: тест показал положительный эффект, но не в ожидаемом объёме.

### Вывод:
- Новый интерфейс действительно улучшает конверсию, но не настолько, как было запланировано.
- Необходимо оценить, оправдан ли такой прирост с точки зрения экономики, затрат на внедрение и влияния на LTV.

Решение о внедрении следует принимать, учитывая не только статистику, но и бизнес-приоритеты.
Если даже +2.51 п.п. даёт ощутимую прибыль — можно рекомендовать к запуску. Если критично именно достижение 3 п.п. — стоит доработать интерфейс и повторить тест.